In [5]:
%%script bash --bg --out stdout
litex_server uart /dev/ttyUSB0 115200

Starting job # 2 in a separate thread.


In [2]:
print(stdout.readlines())

KeyboardInterrupt: 

In [1]:
from litex.soc.tools.remote import RemoteClient
from struct import pack, unpack
import time

In [2]:
for i in range(32):
    try:
        r = RemoteClient(csr_csv="build/csr.csv", debug=False, port=1234 + i)
        r.open()
        print("Connected to Port", 1234 + i)
        break
    except ConnectionRefusedError:
        pass

Connected to Port 1234


In [3]:
print(hex(r.regs.ctrl_scratch.read()))
r.regs.ctrl_scratch.write(0xDEADBEEF)
print(hex(r.regs.ctrl_scratch.read()))

0x12345678
0xdeadbeef


In [4]:
def getId():
    s = ""
    for i in range(64):
        temp = r.read(r.bases.identifier_mem + i * 4)
        if temp == 0:
            break
        s += chr(temp & 0xFF)
    return s
getId()

'Wir trampeln durchs Getreide ... 2019-04-01 00:06:30'

# SPI


    config signal:
        1 offline: all pins high-z (reset=1)
        1 active: cs/transfer active (read-only)
        1 pending: transfer pending in intermediate buffer (read-only)
        1 cs_polarity: active level of chip select (reset=0)
        1 clk_polarity: idle level of clk (reset=0)
        1 clk_phase: first edge after cs assertion to sample data on (reset=0)
            (clk_polarity, clk_phase) == (CPOL, CPHA) in Freescale language.
            (0, 0): idle low, output on falling, input on rising
            (0, 1): idle low, output on rising, input on falling
            (1, 0): idle high, output on rising, input on falling
            (1, 1): idle high, output on falling, input on rising
            There is never a clk edge during a cs edge.
        1 lsb_first: LSB is the first bit on the wire (reset=0)
        1 half_duplex: 3-wire SPI, in/out on mosi (reset=0)
        8 undefined
        8 div_write: counter load value to divide this module's clock
            to generate the SPI write clk (reset=0)
            f_clk/f_spi_write == div_write + 2
        8 div_read: ditto for the read clock

    xfer_config signal:
        16 cs: active high bit mask of chip selects to assert (reset=0)
        6 write_len: 0-M bits (reset=0)
        2 undefined
        6 read_len: 0-M bits (reset=0)
        2 undefined

## Setup SPI

In [4]:
r.regs.spi_config.write(0xFFFF0000)
# 16 bit write transfer (includes read as is 4 wire)
r.regs.spi_xfer.write(0x0010FFFF)

def set_ltc_reg(adr, val):
    word = (0 << 15) | ((adr & 0x7F) << 8) | (val & 0xFF)
    word <<= 16
    r.regs.spi_mosi_data.write(word)
    r.regs.spi_start.write(1)

def get_ltc_reg(adr):
    word = (1 << 15) | ((adr & 0x7F) << 8)
    word <<= 16
    r.regs.spi_mosi_data.write(word)
    r.regs.spi_start.write(1)
    return r.regs.spi_miso_data.read() & 0xFF

In [5]:
for i in range(5):
    print("{:2x}: {:02x}".format(i, get_ltc_reg(i)))

 0: 00
 1: 00
 2: 00
 3: 00
 4: 00


In [112]:
set_ltc_reg(0, 0x80)   # Software reset

In [15]:
set_ltc_reg(1, 0x10)   # Sleep mode

In [16]:
set_ltc_reg(1, 0x00)   # Waky waky

In [158]:
tpValue = 0x1F7
set_ltc_reg(3, (1<<7) | tpValue >> 8)   # Test pattern on + value MSB
set_ltc_reg(4, tpValue & 0xFF)          # Test pattern value LSB

print(hex((get_ltc_reg(3) & 0x3F) << 8 | get_ltc_reg(4)))
print("{:08b}".format(tpValue))

0x1f7
111110111


# FRM clock frequency

In [8]:
#while True:
#    time.sleep(0.5)
print("\r{:9.6f} MHz".format(r.regs.f_frame_value.read() / 1e6), end="")

AttributeError: No such element f_frame_value

In [68]:
"{:08b}".format(0xAF)

'10101111'

In [69]:
"{:08b}".format(0x57)

'01010111'

In [160]:
rVal = r.regs.lvds_clk_peek.read()
print(hex(rVal))
print("{:08b}".format(rVal))

0xf0
11110000


In [166]:
rVal = r.regs.lvds_data_peek.read()
print(hex(rVal))
print("{:08b}".format(rVal))
print("00100011")

0x85
10000101
00100011


In [108]:
r.regs.lvds_bitslip_csr.write(1)

In [400]:
def sign_extend(value, bits):
    sign_bit = 1 << (bits - 1)
    return (value & (sign_bit - 1)) - (value & sign_bit)

In [624]:
a = sign_extend(r.regs.lvds_idelay_a.read(), 9)
b = sign_extend(r.regs.lvds_idelay_b.read(), 9)
print(a, b)

4 0


In [314]:
a = [1, 3, 5, 7, 9, 11, 13, 15]
b = [0, 2, 4, 6, 8, 10, 12, 14]

In [320]:
out = a[::-1] + b[::-1]
out

[15, 13, 11, 9, 7, 5, 3, 1, 14, 12, 10, 8, 6, 4, 2, 0]

In [340]:
def myzip(*vals):
    return [i for t in zip(*vals) for i in t]

In [342]:
myzip(out[:8], out[8:])[::-1]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

In [42]:
r.close()